In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import re
import itertools
import warnings
from joblib import dump

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats, signal
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

warnings.filterwarnings("ignore")
random_state = 42
activities = ["hair", "listen", "sidepump", "dab", "wipetable", "gun", "elbowkick", "pointhigh", "logout"]
n_labels = len(activities)

In [ ]:
dfs = []

def load_data(activity, label, trials, subject):
    for trial in trials:
        df = pd.read_csv(os.path.join("/kaggle/input/human-activity-recognition-dancing/" + subject, activity+str(trial)+".csv"), sep=",", index_col=0)
        df = df[40:-40]
        df["activity"] = [label for _ in range(len(df))]
        df["activityName"] = [activity for _ in range(len(df))]
        df["trial"] = [trial for _ in range(len(df))]
        df["subject"] = [subject for _ in range(len(df))]
        dfs.append(df)

ids = [1, 2]
user = "subject1"
load_data("hair", 0, ids, user)
load_data("listen", 1, ids, user)
load_data("sidepump", 2, ids, user)
load_data("dab", 3, ids, user)
load_data("wipetable", 4, ids, user)
load_data("gun", 5, ids, user)
load_data("elbowkick", 6, ids, user)
load_data("pointhigh", 7, ids, user)
load_data("logout", 8, ids, user)

user = "subject2"
load_data("hair", 0, ids, user)
load_data("listen", 1, ids, user)
load_data("sidepump", 2, ids, user)
load_data("dab", 3, ids, user)
load_data("wipetable", 4, ids, user)
load_data("gun", 5, ids, user)
load_data("elbowkick", 6, ids, user)
load_data("pointhigh", 7, ids, user)
load_data("logout", 8, ids, user)

user = "subject3"
load_data("hair", 0, ids, user)
load_data("listen", 1, ids, user)
load_data("sidepump", 2, ids, user)
load_data("dab", 3, ids, user)
load_data("wipetable", 4, ids, user)
load_data("gun", 5, ids, user)
load_data("elbowkick", 6, ids, user)
load_data("pointhigh", 7, ids, user)
load_data("logout", 8, ids, user)

user = "subject4"
load_data("hair", 0, ids, user)
load_data("listen", 1, ids, user)
load_data("sidepump", 2, ids, user)
load_data("dab", 3, ids, user)
load_data("wipetable", 4, ids, user)
load_data("gun", 5, ids, user)
load_data("elbowkick", 6, ids, user)
load_data("pointhigh", 7, ids, user)
load_data("logout", 8, ids, user)

user = "subject5"
load_data("hair", 0, ids, user)
load_data("listen", 1, ids, user)
load_data("sidepump", 2, ids, user)
load_data("dab", 3, ids, user)
load_data("wipetable", 4, ids, user)
load_data("gun", 5, ids, user)
load_data("elbowkick", 6, ids, user)
load_data("pointhigh", 7, ids, user)
load_data("logout", 8, ids, user)

user = "subject6"
load_data("hair", 0, ids, user)
load_data("listen", 1, ids, user)
load_data("sidepump", 2, ids, user)
load_data("dab", 3, ids, user)
load_data("wipetable", 4, ids, user)
load_data("gun", 5, ids, user)
load_data("elbowkick", 6, ids, user)
load_data("pointhigh", 7, ids, user)
load_data("logout", 8, ids, user)

user = "subject7"
load_data("hair", 0, ids, user)
load_data("listen", 1, ids, user)
load_data("sidepump", 2, ids, user)
load_data("dab", 3, ids, user)
load_data("wipetable", 4, ids, user)
load_data("gun", 5, ids, user)
load_data("elbowkick", 6, ids, user)
load_data("pointhigh", 7, ids, user)
load_data("logout", 8, ids, user)


df = pd.concat(dfs)
print(df.shape)
df.head()

In [ ]:
def plot_distribution_of_data(df, title=""):
    activity, counts = np.unique(df.activityName, return_counts=True)
    index = np.arange(len(activity))
    plt.figure(figsize=(12, 5))
    plt.bar(index, counts, align = 'center', alpha=0.5, color='lightblue')
    plt.suptitle(title, fontsize=16)
    plt.xticks(index, activity, rotation=90, fontsize=8)
    plt.xlim([-.5, 8.5])
    plt.ylabel('frequency', fontsize=10)
    for i, v in enumerate(counts):
        plt.text(i, v, str(v), color='grey', fontweight='bold')
    plt.show()

plot_distribution_of_data(df, title='Class Distribution Of Dataset')

In [ ]:
def plot_distribution_of_data_per_user(df, title=""):
    plt.figure(figsize=(16,8))
    plt.title(title, fontsize=20)
    sns.countplot(x='subject',hue='activityName', data=df)
    plt.show()

plot_distribution_of_data_per_user(df, title='Data Collected Per User')

In [ ]:
def convert_to_timeseries(features, num_time_steps, num_features):
    data = np.reshape(features, newshape=(num_time_steps, num_features))
    return data.T

num_time_steps = 60
X = list()
y = list()
df_len = len(df)
for idx in range(0, df_len, num_time_steps//2):
    window_df = df[idx:idx+num_time_steps]
    labels = window_df["activity"].unique()
    trials = window_df["trial"].unique()
    subjects = window_df["subject"].unique()
    if len(labels) != 1 or len(trials) != 1 or len(subjects) != 1 or len(window_df) < num_time_steps:
        continue
    assert len(labels) == 1 and len(window_df) == num_time_steps
    features = window_df.drop(columns=["activity", "activityName", "subject", "trial"]).values
    features = convert_to_timeseries(features, num_time_steps=num_time_steps, num_features=9)
    X.append(features)
    y.append(labels)

X = np.array(X)
y = np.array(y)

X.shape, y.shape

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion Matrix'):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize=(15,8))
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cmap='Reds'
        print("Normalized Confusion Matrix")
    else:
        cmap='Greens'
        print('Confusion Matrix Without Normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [ ]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            上次验证集损失值改善后等待几个epoch
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            如果是True，为每个验证集损失值改善打印一条信息
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            监测数量的最小变化，以符合改进的要求
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''
        Saves model when validation loss decrease.
        验证损失减少时保存模型。
        '''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model, 'finish_model.pkl')                 # 这里会存储迄今最优的模型
        self.val_loss_min = val_loss

def plot_avg_loss_per_epoch(avg_train_losses, avg_valid_losses):
    # visualize the loss as the network trained
    fig = plt.figure(figsize=(10,8))
    plt.plot(range(1,len(avg_train_losses)+1), avg_train_losses, label='Training Loss')
    plt.plot(range(1,len(avg_valid_losses)+1), avg_valid_losses,label='Validation Loss')

    # find position of lowest validation loss
    minposs = avg_train_losses.index(min(avg_train_losses))+1 
    plt.axvline(minposs, linestyle='--', color='r',label='Early Stopping Checkpoint')

    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.ylim(0, 2) # consistent scale
    plt.xlim(0, len(avg_train_losses)+1) # consistent scale
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

def plot_accuracy_per_epoch(train_accuracies, valid_accuracies):
    # visualize the loss as the network trained
    fig = plt.figure(figsize=(10,8))
    plt.plot(range(1,len(train_accuracies)+1), train_accuracies, label='Train Accuracy')
    plt.plot(range(1,len(valid_accuracies)+1), valid_accuracies,label='Valid Accuracy')

    plt.xlabel('epochs')
    plt.ylabel('accuracy')
    plt.ylim(0, 1) # consistent scale
    plt.xlim(0, len(train_accuracies)+1) # consistent scale
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
class DLTrainer:
    def __init__(self, model, trainloader, validloader, testloader, criterion, optimizer, early_stopping, n_epochs, patience, is_time_series=False):
        self.model = model
        self.trainloader = trainloader
        self.validloader = validloader
        self.testloader = testloader
        
        self.criterion = criterion
        self.optimizer = optimizer
        self.early_stopping = early_stopping
        self.n_epochs = n_epochs
        self.patience = patience
        self.is_time_series = is_time_series

        self.avg_train_losses = []
        self.avg_valid_losses = [] 
        self.train_accuracies = []
        self.valid_accuracies = []

    def train(self):
        self.model.cuda()
        for epoch in range(self.n_epochs):  # loop over the dataset multiple times
            train_correct, train_total, train_loss = self.train_epoch()
            valid_correct, valid_total, valid_loss = self.validate_epoch()

            # calculate average loss over an epoch
            train_accuracy = train_correct / train_total
            valid_accuracy = valid_correct / valid_total

            self.avg_train_losses.append(train_loss)
            self.avg_valid_losses.append(valid_loss)
            self.train_accuracies.append(train_accuracy)
            self.valid_accuracies.append(valid_accuracy)

            log_message = f'[{str(epoch)}/{str(self.n_epochs)}] train_loss: {train_loss:.5f} valid_loss: {valid_loss:.5f} train_accuracy: {train_accuracy:.5f} valid_accuracy: {valid_accuracy:.5f}'
            print(log_message)

            self.early_stopping(valid_loss, self.model)
            
            if self.early_stopping.early_stop:
                break

    def train_epoch(self):
        correct = 0
        total = 0
        losses = []

        self.model.train()
        for data in self.trainloader:
            inputs, labels = data
            inputs = inputs.to("cuda")
            labels = labels.to("cuda")

            self.optimizer.zero_grad()

            outputs = self.model(inputs.float())

            loss = self.criterion(outputs, labels)
            loss.backward()
            self.optimizer.step()

            losses.append(loss.item())

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        return correct, total, np.mean(losses)

    def validate_epoch(self):
        correct = 0
        total = 0
        losses = []

        self.model.eval()
        for data in validloader:
            inputs, labels = data
            inputs = inputs.to("cuda")
            labels = labels.to("cuda")

            outputs = self.model(inputs.float())
            loss = self.criterion(outputs, labels)
            losses.append(loss.item())

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        return correct, total, np.mean(losses)

    def evaluate(self, dataloader):
        correct = 0
        total = 0
        actual = []
        expected = []

        with torch.no_grad():
            for data in dataloader:
                inputs, labels = data
                inputs = inputs.to("cuda")
                labels = labels.to("cuda")
                outputs = self.model(inputs.float())
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                expected += labels.cpu()
                actual += predicted.cpu()

        return correct/total, (expected, actual)

    def show_evaluation(self):
        return self.avg_train_losses, self.avg_valid_losses, self.train_accuracies, self.valid_accuracies


In [ ]:
def compute_mean(data):
    return np.mean(data)

def compute_variance(data):
    return np.var(data)

def compute_median_absolute_deviation(data):
    return stats.median_absolute_deviation(data)

def compute_root_mean_square(data):
    def compose(*fs):
        def wrapped(x):
            for f in fs[::-1]:
                x = f(x)
            return x
        return wrapped
    rms = compose(np.sqrt, np.mean, np.square)
    return rms(data)

def compute_interquartile_range(data):
    return stats.iqr(data)

def compute_percentile_75(data):
    return np.percentile(data, 75)

def compute_kurtosis(data):
    return stats.kurtosis(data)

def compute_min_max(data):
    return np.max(data) - np.min(data)

def compute_signal_magnitude_area(data):
    return np.sum(data) / len(data)

def compute_zero_crossing_rate(data):
    return ((data[:-1] * data[1:]) < 0).sum()

def compute_spectral_centroid(data):
    spectrum = np.abs(np.fft.rfft(data))
    normalized_spectrum = spectrum / np.sum(spectrum)  # like a probability mass function
    normalized_frequencies = np.linspace(0, 1, len(spectrum))
    spectral_centroid = np.sum(normalized_frequencies * normalized_spectrum)
    return spectral_centroid

def compute_spectral_entropy(data):
    freqs, power_density = signal.welch(data)
    return stats.entropy(power_density)

def compute_spectral_energy(data):
    freqs, power_density = signal.welch(data)
    return np.sum(np.square(power_density))

def compute_principle_frequency(data):
    freqs, power_density = signal.welch(data)
    return freqs[np.argmax(np.square(power_density))]

In [ ]:
feature_names = []
for i in ["yaw", "pitch", "row", "gyro_x", "gyro_y", "gyro_z", "acc_x", "acc_y", "acc_z"]:
        for j in ["_mean", "_var", "_mad", "_rms", "_iqr", "_per75", "_kurtosis", "_min_max", "_sma", "_zcr", "_sc", "_entropy", "_energy", "_pfreq"]:
            feature_names.append(i+j)

n_features = len(feature_names)

def rename_raw_data_headers(X, columns):
    X = pd.DataFrame(X)
    X.columns = columns
    return X

def extract_raw_data_features_per_row(f_n):
    f1_mean = compute_mean(f_n)
    f1_var = compute_variance(f_n)
    f1_mad = compute_median_absolute_deviation(f_n)
    f1_rms = compute_root_mean_square(f_n)
    f1_iqr = compute_interquartile_range(f_n)
    f1_per75 = compute_percentile_75(f_n)
    f1_kurtosis = compute_kurtosis(f_n)
    f1_min_max = compute_min_max(f_n)
    f1_sma = compute_signal_magnitude_area(f_n)
    f1_zcr = compute_zero_crossing_rate(f_n)
    f1_sc = compute_spectral_centroid(f_n)
    f1_entropy = compute_spectral_entropy(f_n)
    f1_energy = compute_spectral_energy(f_n)
    f1_pfreq = compute_principle_frequency(f_n)
    return f1_mean, f1_var, f1_mad, f1_rms, f1_iqr, f1_per75, f1_kurtosis, f1_min_max, f1_sma, f1_zcr, f1_sc, f1_entropy, f1_energy, f1_pfreq

def extract_raw_data_features(X):
    new_features = np.ones((X.shape[0], n_features))
    rows = X.shape[0]
    cols = X.shape[1]

    for row in range(rows):
        features = []
        for col in range(cols):
            f_n = X[row][col]
            feature = extract_raw_data_features_per_row(f_n)
            features.extend(feature)
        new_features[row] = np.array(features)

    return new_features


def prepare_raw_data_dataframe(X, y, random_state=42):
    X = extract_raw_data_features(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=random_state)
    X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=random_state)

    return X_train, X_valid, X_test, y_train, y_valid, y_test

X_train, X_valid, X_test, y_train, y_valid, y_test = prepare_raw_data_dataframe(X, y)

In [ ]:
scaler = preprocessing.StandardScaler()

def scale_data(data, is_train=False):
    if is_train:
        data = scaler.fit_transform(data)
    else:
        data = scaler.transform(data)
    return data

X_train = scale_data(X_train, is_train=True)
X_valid = scale_data(X_valid)
X_test = scale_data(X_test)
X_train.shape

In [ ]:
df2 = pd.DataFrame(X_train, columns=feature_names)
df2["activityName"] = y_train
df2["activityName"] = df2["activityName"].replace([0,1,2,3,4,5,6,7,8], activities)
df2

In [ ]:
exploratory_features = ["acc_z_min_max", "pitch_pfreq", "acc_z_kurtosis"]

def plot_boxplot_activity_distribution(data, y):
    sns.boxplot(x='activityName', y=y, data=data)
    plt.axhline(y=0.08, xmin=0.1, xmax=0.9,c='m',dashes=(5,3))
    plt.title(feature, fontsize=15)
    plt.xticks(rotation = 40)
    plt.show()

for feature in exploratory_features:
    plot_boxplot_activity_distribution(df2, feature)

In [ ]:
perplexities = [2,5,10,20,50]

def plot_tsne(X_data, y_data, perplexity, n_iter=1000, img_name_prefix='t-sne'):
    # perform t-sne
    X_reduced = TSNE(verbose=2, perplexity=perplexity).fit_transform(X_data)
    
    # prepare the data for seaborn         
    df = pd.DataFrame({'x':X_reduced[:,0], 'y':X_reduced[:,1] ,'label':y_data})
    
    # draw the plot in appropriate place in the grid
    sns.lmplot(data=df, x='x', y='y', hue='label', fit_reg=False, size=8,\
                palette="Set1")
    plt.title("perplexity : {} and max_iter : {}".format(perplexity, n_iter))
    img_name = img_name_prefix + '_perp_{}_iter_{}.png'.format(perplexity, n_iter)
    plt.show()

for perplexity in perplexities:
    X_pre_tsne = df2.drop(columns=['activityName'], axis=1)
    y_pre_tsne = df2['activityName']
    plot_tsne(X_data=X_pre_tsne, y_data=y_pre_tsne, perplexity=perplexity)

In [ ]:
class DNNDataset(object):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __getitem__(self, idx):
        data = self.X[idx]
        target = self.y[idx][0] 
        return data, target

    def __len__(self):
        return len(self.X)

def prepare_dnn_dataloader(X_train, X_valid, X_test, y_train, y_valid, y_test):
    traindataset = DNNDataset(X_train, y_train)

    trainloader = torch.utils.data.DataLoader(
        traindataset, 
        batch_size=100, 
        shuffle=True, 
        num_workers=4,
    )

    validdataset = DNNDataset(X_valid, y_valid)

    validloader = torch.utils.data.DataLoader(
        validdataset, 
        batch_size=100, 
        shuffle=True, 
        num_workers=4,
    )

    testdataset = DNNDataset(X_test, y_test)

    testloader = torch.utils.data.DataLoader(
        testdataset, 
        batch_size=100, 
        shuffle=True, 
        num_workers=4,
    )

    return trainloader, validloader, testloader

In [ ]:
class DNN(nn.Module):

    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(126, 64)
        self.dp1 = nn.Dropout(0.1)

        self.fc2 = nn.Linear(64, 16)
        self.dp2 = nn.Dropout(0.1)
        
        self.fc3 = nn.Linear(16,n_labels)

    def forward(self, x):
        x = self.fc1(x)
        x = self.dp1(x)

        x = self.fc2(x)
        x = self.dp2(x)

        x = self.fc3(x)
        return x

trainloader, validloader, testloader = prepare_dnn_dataloader(X_train, X_valid, X_test, y_train, y_valid, y_test)
inputs, labels = next(iter(trainloader))
model = DNN()
outputs = model(inputs.float())
labels[:1], outputs[:1]

In [ ]:
print(model)

In [ ]:
patience = 10
n_epochs = 1000

model = DNN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
early_stopping = EarlyStopping(patience=patience)
trainloader, validloader, testloader = prepare_dnn_dataloader(X_train, X_valid, X_test, y_train, y_valid, y_test)

trainer = DLTrainer(model, trainloader, validloader, testloader, criterion, optimizer, early_stopping, n_epochs, patience)
trainer.train()

In [ ]:
dnn_valid_accuracy, dnn_valid_prediction = trainer.evaluate(validloader)
dnn_test_accuracy, dnn_test_prediction = trainer.evaluate(testloader)
print(dnn_valid_accuracy, dnn_test_accuracy)

In [ ]:
cnf_matrix = confusion_matrix(*dnn_test_prediction)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure(figsize=(15,8))
plot_confusion_matrix(cnf_matrix, classes=activities, title='DNN Testing Confusion Matrix Without Normalization')

# Plot normalized confusion matrix
plt.figure(figsize=(15,8))
plot_confusion_matrix(cnf_matrix, classes=activities, normalize=True, title='DNN Testing Normalized Confusion Matrix')

plt.show()

In [ ]:
avg_train_losses, avg_valid_losses, train_accuracies, valid_accuracies = trainer.show_evaluation()
plot_avg_loss_per_epoch(avg_train_losses, avg_valid_losses)
plot_accuracy_per_epoch(train_accuracies, valid_accuracies)

In [ ]:
torch.save(model.state_dict(), "dnn_model.pth")
dump(scaler, 'dnn_std_scaler.bin', compress=True)

In [ ]:
dfs = []

ids = [3,]
user = "subject1"
load_data("hair", 0, ids, user)
load_data("listen", 1, ids, user)
load_data("sidepump", 2, ids, user)
load_data("dab", 3, ids, user)
load_data("wipetable", 4, ids, user)
load_data("gun", 5, ids, user)
load_data("elbowkick", 6, ids, user)
load_data("pointhigh", 7, ids, user)
load_data("logout", 8, ids, user)

user = "subject2"
load_data("hair", 0, ids, user)
load_data("listen", 1, ids, user)
load_data("sidepump", 2, ids, user)
load_data("dab", 3, ids, user)
load_data("wipetable", 4, ids, user)
load_data("gun", 5, ids, user)
load_data("elbowkick", 6, ids, user)
load_data("pointhigh", 7, ids, user)
load_data("logout", 8, ids, user)

user = "subject3"
load_data("hair", 0, ids, user)
load_data("listen", 1, ids, user)
load_data("sidepump", 2, ids, user)
load_data("dab", 3, ids, user)
load_data("wipetable", 4, ids, user)
load_data("gun", 5, ids, user)
load_data("elbowkick", 6, ids, user)
load_data("pointhigh", 7, ids, user)
load_data("logout", 8, ids, user)

user = "subject4"
load_data("hair", 0, ids, user)
load_data("listen", 1, ids, user)
load_data("sidepump", 2, ids, user)
load_data("dab", 3, ids, user)
load_data("wipetable", 4, ids, user)
load_data("gun", 5, ids, user)
load_data("elbowkick", 6, ids, user)
load_data("pointhigh", 7, ids, user)
load_data("logout", 8, ids, user)

user = "subject5"
load_data("hair", 0, ids, user)
load_data("listen", 1, ids, user)
load_data("sidepump", 2, ids, user)
load_data("dab", 3, ids, user)
load_data("wipetable", 4, ids, user)
load_data("gun", 5, ids, user)
load_data("elbowkick", 6, ids, user)
load_data("pointhigh", 7, ids, user)
load_data("logout", 8, ids, user)

user = "subject6"
load_data("hair", 0, ids, user)
load_data("listen", 1, ids, user)
load_data("sidepump", 2, ids, user)
load_data("dab", 3, ids, user)
load_data("wipetable", 4, ids, user)
load_data("gun", 5, ids, user)
load_data("elbowkick", 6, ids, user)
load_data("pointhigh", 7, ids, user)
load_data("logout", 8, ids, user)

user = "subject7"
load_data("hair", 0, ids, user)
load_data("listen", 1, ids, user)
load_data("sidepump", 2, ids, user)
load_data("dab", 3, ids, user)
load_data("wipetable", 4, ids, user)
load_data("gun", 5, ids, user)
load_data("elbowkick", 6, ids, user)
load_data("pointhigh", 7, ids, user)
load_data("logout", 8, ids, user)


df = pd.concat(dfs)
df.shape

In [ ]:
X = list()
y = list()
df_len = len(df)
for idx in range(0, df_len, num_time_steps//2):
    window_df = df[idx:idx+num_time_steps]
    labels = window_df["activity"].unique()
    trials = window_df["trial"].unique()
    subjects = window_df["subject"].unique()
    if len(labels) != 1 or len(trials) != 1 or len(subjects) != 1 or len(window_df) < num_time_steps:
        continue
    assert len(labels) == 1 and len(window_df) == num_time_steps
    features = window_df.drop(columns=["activity", "activityName", "subject", "trial"]).values
    features = convert_to_timeseries(features, num_time_steps=num_time_steps, num_features=9)
    X.append(features)
    y.append(labels)

X = np.array(X)
y = np.array(y)
X = extract_raw_data_features(X)
X = scale_data(X)
X.shape, y.shape

In [ ]:
dataset = DNNDataset(X, y)

dataloader = torch.utils.data.DataLoader(
    dataset, 
    batch_size=1, 
    shuffle=True, 
    num_workers=4,
)

In [ ]:
dnn_test_accuracy, dnn_test_prediction = trainer.evaluate(dataloader)
dnn_test_accuracy

In [ ]:
cnf_matrix = confusion_matrix(*dnn_test_prediction)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure(figsize=(15,8))
plot_confusion_matrix(cnf_matrix, classes=activities, title='DNN Testing Confusion Matrix Without Normalization')

# Plot normalized confusion matrix
plt.figure(figsize=(15,8))
plot_confusion_matrix(cnf_matrix, classes=activities, normalize=True, title='DNN Testing Normalized Confusion Matrix')

plt.show()